# Video analysis


## Setting
 - Auto Reload
 - path for utils

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys, os
module_path = "../.."
sys.path.append(os.path.abspath(module_path))

## 1. Create Bedrock client

In [ ]:
from pprint import pprint
from termcolor import colored
from utils import bedrock
from utils.bedrock import bedrock_info

### ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----
- os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
- os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
- os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."
- os.environ["BEDROCK_ENDPOINT_URL"] = "<YOUR_ENDPOINT_URL>"  # E.g. "https://..."

In [ ]:
boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    endpoint_url=os.environ.get("BEDROCK_ENDPOINT_URL", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None),
)

print (colored("\n== FM lists ==", "green"))
pprint (bedrock_info.get_list_fm_models(verbose=False))

## 2. LLM 정의

In [ ]:
from utils.bedrock import bedrock_model
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

In [ ]:
llm = bedrock_model(
    model_id=bedrock_info.get_model_id(model_name="Claude-V3-5-Sonnet"),
    bedrock_client=boto3_bedrock,
    stream=True,
    callbacks=[StreamingStdOutCallbackHandler()],
    inference_config={
        'maxTokens': 1024,
        'stopSequences': ["\n\nHuman"],
        'temperature': 0.01,
        #'topP': ...,
    }
    #additional_model_request_fields={"top_k": 200}
)

## 3. Analysis

In [ ]:
from textwrap import dedent
from utils.bedrock import bedrock_utils, bedrock_chain

class llm_call():

    def __init__(self, **kwargs):

        self.llm=kwargs["llm"]
        self.verbose = kwargs.get("verbose", False)
        self.chain = bedrock_chain(bedrock_utils.converse_api) | bedrock_chain(bedrock_utils.outputparser)

    def _message_format(self, role, message):

        if role == "user":
             message_format = {
                "role": "user",
                "content": [{"text": dedent(message)}]
            }
        elif role == "assistant":
            
            message_format = {
                "role": "assistant",
                'content': [{'text': dedent(message)}]
            }

        return message_format
            
    def invoke(self, **kwargs):

        system_prompts = kwargs.get("system_prompts", None)
        messages = kwargs["messages"]
        #llm_name = kwargs["llm_name"]
    
        response = self.chain( ## pipeline의 제일 처음 func의 argument를 입력으로 한다. 여기서는 converse_api의 arg를 쓴다.
            llm=self.llm,
            system_prompts=system_prompts,
            messages=messages,
            verbose=self.verbose
        )
        
        ai_message = self._message_format(role="assistant", message=response["text"])
        messages.append(ai_message)
        return response, messages

In [ ]:
llm_caller = llm_call(llm=llm, verbose=True)

In [ ]:
analyzer = video_analyzer(
    llm=llm,
    video_path = "./video/video_sample.mp4"
)

### 3.2 Node tester

#### 3.2.1.sample_video_frames

In [ ]:
import cv2
import os
import shutil
from typing import Tuple, Optional
import matplotlib.pyplot as plt

In [ ]:
def setup_directory(dir_path):
    
    print ("===setup_directory===")
    # 디렉토리가 존재하는지 확인
    if os.path.exists(dir_path):
        # 존재하면 삭제
        shutil.rmtree(dir_path)
        print(f"기존 디렉토리 삭제됨: {dir_path}")
    
    # 디렉토리 생성
    os.makedirs(dir_path)
    print(f"디렉토리 생성됨: {dir_path}")
    print ("=====================")
    
def sample_video_frames(video_path: str, sample_msec: int, output_dir: Optional[str] = None) -> Tuple[int, int]:

    print ("===sample_video_frames===")
    """
    비디오에서 특정 시간 간격으로 프레임을 샘플링하는 함수

    Args:
        video_path (str): 비디오 파일 경로
        sample_msec (int): 샘플링 간격 (밀리초)
        output_dir (Optional[str]): 프레임 저장 디렉토리. None이면 저장하지 않음

    Returns:
        Tuple[int, int]: (총 프레임 수, 샘플링된 프레임 수)

    Raises:
        FileNotFoundError: 비디오 파일이 없는 경우
        ValueError: 샘플링 간격이 잘못된 경우
    """
    # 입력값 검증
    if not os.path.exists(video_path):
        raise FileNotFoundError(f"비디오 파일을 찾을 수 없습니다: {video_path}")

    if sample_msec <= 0:
        raise ValueError("샘플링 간격은 0보다 커야 합니다")

    # 비디오 캡처 객체 생성
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise RuntimeError("비디오 파일을 열 수 없습니다")

    # 비디오 정보 가져오기
    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # 샘플링 간격(프레임 단위) 계산
    frame_interval = max(1, int(sample_msec / 1000 * fps))

    # 출력 디렉토리 생성 (지정된 경우)
    if output_dir is not None:
        setup_directory(output_dir)

    sampled_count = 0
    frame_count = 0
    sampled_frame = {"frame": [], "seq": []}

    while True:
        ret, frame = cap.read()
        if not ret: break

        # frame_interval마다 프레임 처리
        if frame_count % frame_interval == 0:
            if output_dir is not None:
                # 프레임 저장
                frame_path = os.path.join(output_dir, f"frame_{frame_count:06d}.jpg")
                cv2.imwrite(frame_path, frame)
            sampled_count += 1
            sampled_frame["frame"].append(frame)
            sampled_frame["seq"].append(frame_count)
        frame_count += 1

        # 진행상황 출력 (10% 단위)
        if frame_count % (total_frames // 10) == 0:
            progress = (frame_count / total_frames) * 100
            print(f"진행률: {progress:.1f}%")

    # 자원 해제
    cap.release()

    print(f"\n처리 완료:")
    print(f"총 프레임 수: {total_frames}")
    print(f"프레임 크기: {sampled_frame['frame'][0].shape[1]}X{sampled_frame['frame'][0].shape[0]}")
    print(f"샘플링된 프레임 수: {sampled_count}")
    print(f"샘플링 간격: {frame_interval}프레임 ({sample_msec}ms)")
    if output_dir is not None:
        print(f"저장 위치: {output_dir}")

    print ("=========================")

    return sampled_frame, total_frames, sampled_count

In [ ]:
# 프레임 저장하면서 샘플링
sampled_frames, total_frame_cnt, sampled_cnt = sample_video_frames(
    video_path="./video/video_sample.mp4",
    sample_msec=1000,  # 100ms(0.1초)마다 샘플링
    output_dir="./workplace"
)

#### 3.2.2.summary_frames

In [ ]:
def _frame_to_bytes(frame, format='.png'):
    """
    cv2 frame을 bytes로 변환
    
    Args:
        frame: cv2로 읽은 이미지/프레임
        format: 이미지 포맷 (예: '.jpg', '.png')
    
    Returns:
        bytes: 이미지의 바이트 데이터
    """
    # imencode() 함수로 프레임을 지정된 포맷의 이미지로 인코딩
    # 반환값: (success, encoded_image)
    success, buffer = cv2.imencode(format, frame)
    
    if not success:
        raise ValueError("이미지 인코딩 실패")
    
    # numpy array를 bytes로 변환
    return buffer.tobytes()

def _get_message_from_string(role, string, imgs=None):
        
        message = {
            "role": role,
            "content": [{"text": dedent(string)}]
        }
        
        if imgs is not None:
            for img in imgs:
                img_message = {
                    "image": {
                        "format": 'png',
                        "source": {"bytes": img}
                    }
                }
                message["content"].append(img_message)

        return message



In [ ]:
'''
<input>
1. frame: CCTV에서 캡처된 단일 프레임 이미지
2. frame_info: 
   - timestamp: 프레임이 캡처된 시간 정보
   - location: CCTV가 설치된 장소 정보
   - frame_number: 전체 시퀀스에서 현재 프레임의 순서
</input>
'''

In [ ]:
messages = []
def summary_frames(**kwargs):
    sampled_frames=kwargs["sampled_frames"]
    total_frame_cnt=kwargs["total_frame_cnt"]
    
    system_prompts = dedent(
        '''
        당신은 CCTV 영상 분석 전문가입니다.
        주어진 CCTV 프레임 이미지를 분석하고 자연어로 상황을 설명하는 것이 당신의 임무입니다.

        <task>
        CCTV 프레임 이미지를 관찰하고 해당 장면에서 발생하는 상황을 자연어로 설명
        </task>

        <input>
        1. frame: CCTV에서 캡처된 단일 프레임 이미지
        2. frame_info: 
           - frame_number: 전체 시퀀스에서 현재 프레임의 순서
           - total_frame_number: 전체 프레임 수
        </input>

        <output_format>
        JSON 형식으로 다음 형태로 응답하세요:
        {
            "scene_description": "현재 프레임에서 관찰되는 상황에 대한 객관적 설명",
            "key_elements": {
                "static_objects": ["장면에서 고정된 물체들의 리스트"],
                "moving_objects": ["움직이는 물체/사람들의 리스트"],
                "activities": ["발생하는 활동들의 리스트"]
            },
            "frame_seq": "frame_number"
        }
        </output_format>

        <instruction>
        1. 주어진 프레임을 객관적으로 관찰하세요.
        2. 장면에서 고정된 물체들의 위치와 상태를 파악하세요.
        3. 움직이는 물체나 사람들의 활동을 식별하세요.
        4. 관찰된 모든 활동을 시간 순서대로 설명하세요.
        5. 특이사항이나 중요한 변화가 있다면 이를 강조하세요.
        6. 추측이나 주관적 해석은 최소화하고 관찰 가능한 사실만 설명하세요.
        7. 시간 정보를 포함하여 맥락을 제공하세요.
        8. 설명은 간결하고 명확하게 작성하세요.
        9. 설명은 한글로 작성하세요.
        </instruction>

        <consideration>
        1. 움직임이 없는 물체/사람에 대해서는 설명하지 마세요.
        2. 사람이나 물체의 위치 변화에 특히 주의를 기울이세요.
        3. 시야가 가려지거나 불명확한 부분이 있다면 이를 명시하세요.
        4. 보안과 프라이버시를 고려하여 개인을 특정할 수 있는 세부 정보는 제외하세요.
        5. 여러 물체나 사람이 있는 경우, 각각을 구분하여 설명하세요.
        6. 비정상적이거나 특이한 활동이 관찰되면 이를 강조하세요.
        7. 조명 상태나 화질로 인한 제약사항이 있다면 이를 언급하세요.
        </consideration>
        
        '''
    )
    user_prompts = dedent(
        '''
        This is <frame_number>{frame_number}</frame_number> and <total_frame_number>{total_frame_number}</total_frame_number>
        This is the frame.
        
        '''
    )
    
    img_bytes = _frame_to_bytes(sampled_frames["frame"][20])
    system_prompts = bedrock_utils.get_system_prompt(system_prompts=system_prompts)  
    
    context = {
        "frame_number": sampled_frames["seq"][20],
        "total_frame_number": total_frame_cnt,
    }
    user_prompts = user_prompts.format(**context)
        
    message = _get_message_from_string(role="user", string=user_prompts, imgs=[img_bytes])
    messages.append(message)
    #print (messages)

    resp, messages_updated = llm_caller.invoke(messages=messages, system_prompts=system_prompts)
    
    rgb_frame = cv2.cvtColor(sampled_frames["frame"][20], cv2.COLOR_BGR2RGB)

    # 이미지 표시
    plt.figure(figsize=(10, 6))
    plt.imshow(rgb_frame)
    plt.axis('off')  # 축 숨기기
    plt.show()
    

In [ ]:
summary_frames(
    sampled_frames=sampled_frames,
    total_frame_cnt=total_frame_cnt
)

In [ ]:
class video_analyzer():

    def __init__(self, **kwargs):

        self.llm=kwargs["llm"]
        self.video_path=kwargs["video_path"]
        #self.state = GraphState

        self.llm_caller = llm_call(
            llm=self.llm,
            verbose=False
        ) 

        self._graph_definition()
        self.messages = []
        self.img_bytes = ""

        #self.timer = TimeMeasurement()

    def _get_string_from_message(self, message):
        return message["content"][0]["text"]

    def _get_message_from_string(self, role, string, img=None):
        
        message = {
            "role": role,
            "content": [{"text": dedent(string)}]
        }
        
        if img is not None:
            img_message = {
                "image": {
                    "format": 'png',
                    "source": {"bytes": img}
                }
            }
            message["content"].append(img_message)

        return message

    def _png_to_bytes(self, file_path):
        try:
            with open(file_path, "rb") as image_file:
                # 파일을 바이너리 모드로 읽기
                binary_data = image_file.read()
                
                # 바이너리 데이터를 base64로 인코딩
                base64_encoded = base64.b64encode(binary_data)
                
                # bytes 타입을 문자열로 디코딩
                base64_string = base64_encoded.decode('utf-8')
                
                return binary_data, base64_string
                
        except FileNotFoundError:
            return "Error: 파일을 찾을 수 없습니다."
        except Exception as e:
            return f"Error: {str(e)}"

    def show_image(base64_string):
        try:
            # base64 문자열을 디코딩하여 바이너리 데이터로 변환
            image_data = base64.b64decode(base64_string)
            
            # 바이너리 데이터를 이미지로 변환
            image = Image.open(io.BytesIO(image_data))
            
            # matplotlib을 사용하여 이미지 표시
            plt.imshow(image)
            plt.axis('off')  # 축 제거
            plt.show()
        except Exception as e:
            print(f"Error: 이미지를 표시하는 데 실패했습니다. {str(e)}")

    def get_messages(self, ):
        return self.messages
        
    def _graph_definition(self, **kwargs):

        def sample_video_frames(video_path: str, sample_msec: int, output_dir: Optional[str] = None) -> Tuple[int, int]:

            print ("===sample_video_frames===")
            """
            비디오에서 특정 시간 간격으로 프레임을 샘플링하는 함수

            Args:
                video_path (str): 비디오 파일 경로
                sample_msec (int): 샘플링 간격 (밀리초)
                output_dir (Optional[str]): 프레임 저장 디렉토리. None이면 저장하지 않음

            Returns:
                Tuple[int, int]: (총 프레임 수, 샘플링된 프레임 수)

            Raises:
                FileNotFoundError: 비디오 파일이 없는 경우
                ValueError: 샘플링 간격이 잘못된 경우
            """
            # 입력값 검증
            if not os.path.exists(video_path):
                raise FileNotFoundError(f"비디오 파일을 찾을 수 없습니다: {video_path}")

            if sample_msec <= 0:
                raise ValueError("샘플링 간격은 0보다 커야 합니다")

            # 비디오 캡처 객체 생성
            cap = cv2.VideoCapture(video_path)
            if not cap.isOpened():
                raise RuntimeError("비디오 파일을 열 수 없습니다")

            # 비디오 정보 가져오기
            fps = cap.get(cv2.CAP_PROP_FPS)
            total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

            # 샘플링 간격(프레임 단위) 계산
            frame_interval = max(1, int(sample_msec / 1000 * fps))

            # 출력 디렉토리 생성 (지정된 경우)
            if output_dir is not None:
                setup_directory(output_dir)

            sampled_count = 0
            frame_count = 0
            sampled_frame = []

            while True:
                ret, frame = cap.read()
                if not ret: break

                # frame_interval마다 프레임 처리
                if frame_count % frame_interval == 0:
                    if output_dir is not None:
                        # 프레임 저장
                        frame_path = os.path.join(output_dir, f"frame_{frame_count:06d}.jpg")
                        cv2.imwrite(frame_path, frame)
                    sampled_count += 1
                    sampled_frame.append(frame)
                frame_count += 1

                # 진행상황 출력 (10% 단위)
                if frame_count % (total_frames // 10) == 0:
                    progress = (frame_count / total_frames) * 100
                    print(f"진행률: {progress:.1f}%")

            # 자원 해제
            cap.release()

            print(f"\n처리 완료:")
            print(f"총 프레임 수: {total_frames}")
            print(f"프레임 크기: {sampled_frame[0].shape[1]}X{sampled_frame[0].shape[0]}")
            print(f"샘플링된 프레임 수: {sampled_count}")
            print(f"샘플링 간격: {frame_interval}프레임 ({sample_msec}ms)")
            if output_dir is not None:
                print(f"저장 위치: {output_dir}")

            print ("=========================")

            return sampled_frame, total_frames, sampled_count
        
        def agent(state):

            self.timer.start()
            self.timer.reset()

            print("---CALL AGENT---")
            ask = state["ask"]

            """
            현재 상태를 기반으로 에이전트 모델을 호출하여 응답을 생성합니다. 질문에 따라 검색 도구를 사용하여 검색을 결정하거나 단순히 종료합니다.
        
            Args:
                state (messages): 현재 상태
        
            Returns:
                state (messages): 현재 상태 메시지에 에이전트 응답이 추가된 업데이트된 상태
            """

            system_prompts = dedent(
                '''
                <task>
                사용자 메시지를 분석하여 차트 생성 여부를 결정하는 에이전트 역할 수행
                </task>
                
                <instruction>
                1. 사용자 메시지를 주의 깊게 분석하세요.
                2. 차트, 그래프, 데이터 시각화와 관련된 키워드를 찾으세요.
                3. 수치 데이터나 통계 정보의 존재 여부를 확인하세요.
                4. 분석 결과를 바탕으로 차트 생성 필요성을 판단하세요.
                5. 주어진 데이터 (dataset) 및 컬럼 정보 (column_info)를 참고하여 생성 가능 여부 또한 고려하세요.
                6. 데이터로부터 답변할 수 없는 요청을 한다면 최종 결정을 "END"로 하세요.
                7. 판단이 모호한 경우, 사용자에게 직접 차트 생성 의도를 물어보세요.
                </instruction>
                
                <consideration>
                - 사용자의 의도를 정확히 파악하는 것이 중요합니다.
                - 명시적인 차트 요청이 없더라도 데이터 시각화가 유용할 수 있는 상황을 고려하세요.
                - 단순한 질문이나 대화 종료 요청은 차트 생성이 불필요할 수 있습니다.
                - 기존 요청 결과에 대한 추가사항이라고 판단되면, 추가 코드 생성을 하지 말고 "GENERATE_CHART"를 출력하세요.
                </consideration>
                
                <output_format>
                결정에 따라 다음 중 하나를 출력하세요. 반드시 아래 2개중 1개를 선택합니다.:
                1. "GENERATE_CHART (간단한 이유)" - 차트 생성이 필요한 경우
                2. "END (간단한 이유)" - 차트 생성이 할 수 없거나 대화를 종료해야 하는 경우
                
                예시:
                GENERATE_CHART (사용자가 연간 수익 추이 그래프 요청)
                END (단순한 날씨 질문으로 차트 불필요)
                </output_format>

                This is the <request>{request}</request>
                
                '''
            )

            context = {
                "request": str(self.df.sample(10, random_state=0).to_csv()),
                "column_info": str(self.column_info.to_csv())
            }
            system_prompts = system_prompts.format(**context)
            system_prompts = bedrock_utils.get_system_prompt(system_prompts=system_prompts)

            

            message = self._get_message_from_string(role="user", string=ask)
            self.messages.append(message)

            resp, messages_updated = self.llm_caller.invoke(messages=self.messages, system_prompts=system_prompts, llm_name="sonnet")
            self.messages = messages_updated
            
            return self.state(ask=ask, prev_node="AGENT")

#         def should_chart_generation(state):
#             """
#             에이전트가 차트를 생성하는데 있어 추가적으로 고려해야 하는 상황이 있는지 결정합니다.
        
#             이 함수는 상태의 마지막 메시지에서 함수 호출을 확인합니다. 함수 호출이 있으면 정보 검색 프로세스를 계속합니다. 그렇지 않으면 프로세스를 종료합니다.
        
#             Args:
#                 state (messages): 현재 상태
        
#             Returns:
#                 str: 검색 프로세스를 "계속"하거나 "종료"하는 결정
#             """
        
#             print("\n---DECIDE TO CHART GENERATION---")
#             #messages = state["messages"]
#             last_message = self._get_string_from_message(self.messages[-1])
            
#             # 함수 호출이 없으면 종료합니다.
#             if "GENERATE_CHART" not in last_message:
#                 print("---DECISION: DO NOT CHART GENERATION / DONE---")
#                 return "end"
#             # 그렇지 않으면 함수 호출이 있으므로 계속합니다.
#             else:
#                 print("---DECISION: CHART GENERATION---")
#                 return "continue"

#         def ask_reformulation(state):

#             print("---ASK REFORMULATION---")
#             ask = state["ask"]

#             system_prompts = dedent(
#                 '''
#                 당신은 사용자의 텍스트 요청을 분석하여 중요한 정보를 추출하는 전문가입니다.
#                 주어진 structured dataset(df)에 대한 분석 요청(request)을 처리하는 것이 당신의 주요 임무입니다.

#                 <task>
#                 1. 사용자의 텍스트 요청에서 분석 대상이 되는 target app의 이름을 식별하고 추출하세요.
#                 2. 사용자의 구체적인 분석 요청 사항을 분석하고, 필요하다면 결과를 차트로 표현하기 적합현 형태로 요청 사항을 수정해 주세요.
#                 </task>
                
#                 <output_format>
#                 JSON 형식으로 다음 정보를 포함하여 응답하세요:
#                 {{
#                   "target_apps": ["추출된 앱 이름"],
#                   "ask_reformulation": "파악된 분석 요청 사항"
#                 }}
#                 </output_format>

#                 <instruction>
#                 - target app이 명시적으로 언급되지 않은 경우, "target_app" 필드를 "unspecified"로 설정하세요.
#                 - target app이 복수 개인 경우, list 형태로 모두 언급하세요. 예를 들자면 ["앱 이름 1", "앱 이름 2"]로 표현합니다. 
#                 - 분석 요청이 불명확한 경우, 가능한 한 사용자의 의도를 추론하여 "ask_reformulation" 필드를 작성하세요.
#                 - 추출 및 파악한 정보만을 간결하게 제공하고, 추가적인 설명이나 해석은 하지 마세요.
#                 </instruction>

#                 이 정보를 바탕으로 다음 노드가 적절한 분석을 수행할 수 있도록 정확하고 명확한 정보를 제공하는 것이 중요합니다.
#                 '''
#             )
#             system_prompts = bedrock_utils.get_system_prompt(system_prompts=system_prompts)

#             user_prompts = dedent(
#                 '''
#                 This is the result of `print(df.head())`: <dataset>{dataset}</dataset>
#                 Here is the column information in detail, this is the results of `print(column_info)`: <column_info>{column_info}</column_info>
#                 Here is user's request: <request>{ask}</request>
#                 '''
#             )
#             context = {
#                 "dataset": str(self.df.sample(10, random_state=0).to_csv()),
#                 "column_info": str(self.column_info.to_csv()),
#                 "ask": ask
#             }
#             user_prompts = user_prompts.format(**context)
            
#             message = self._get_message_from_string(role="user", string=user_prompts)            
#             self.messages.append(message)

#             resp, messages_updated = self.llm_caller.invoke(messages=self.messages, system_prompts=system_prompts, llm_name="sonnet")

#             results = eval(resp['text'])
#             target_apps, ask_reformulation = results["target_apps"], results["ask_reformulation"]
#             self.messages=messages_updated

#             return self.state(target_apps=target_apps, ask_refo=ask_reformulation, prev_node="ASK_REFORMULATION")

#         def code_generation_for_chart(state):

#             print("---CODE GENERATION FOR CHART---")
#             ask_reformulation = state["ask_refo"]
#             previous_node = state["prev_node"]
#             code_error = state["code_err"]

#             system_prompts = dedent(
#                 '''
#                 당신은 데이터 분석과 시각화 전문가입니다.
#                 주어진 structured dataset, dataset의 컬럼 정보, 그리고 사용자의 분석 요청사항을 바탕으로 적절한 차트를 생성하는 Python 코드를 작성하는 것이 당신의 임무입니다.

#                 <task>
#                 사용자의 요청에 적합한 차트생성 python 코드 작성
#                 </task>

#                 <input>
#                 1. dataset: 분석할 데이터셋
#                 2. column_info: 각 컬럼의 이름과 데이터 타입
#                 3. question: 어떤 분석을 원하는지에 대한 설명
#                 </input>
                
#                 <output_format>
#                 JSON 형식으로 다음 형태로 응답하세요. 절대 JSON 포멧 외 텍스트는 넣지 마세요.:
#                 {{
#                     "code": """사용자의 요청을 충족시키는 차트를 생성하는 Python 코드"""
#                     "img_path": """생성된 차트의 저장 경로"""
#                 }}
#                 </output_format>

#                 <instruction>
#                 1. 데이터셋과 컬럼 정보를 신중히 분석하세요.
#                 2. 사용자의 분석 요청사항을 정확히 이해하세요.
#                 3. 요청사항에 가장 적합한 차트 유형을 선택하세요 (예: 막대 그래프, 선 그래프, 산점도, 파이 차트 등).
#                 4. 선택한 차트 유형에 맞는 Python 라이브러리를 사용하세요 (예: matplotlib, seaborn, plotly 등).
#                 5. 데이터 전처리가 필요한 경우 pandas를 사용하여 데이터를 적절히 가공하세요.
#                 6. 차트의 제목, 축 레이블, 범례 등을 명확하게 설정하세요.
#                 7. 필요한 경우 차트의 색상, 스타일, 크기 등을 조정하여 가독성을 높이세요.
#                 8. 코드 실행 시 발생할 수 있는 예외 상황을 고려하여 적절한 예외 처리를 포함하세요.
#                 9. 생성된 차트를 저장하거나 표시하는 코드를 포함하세요.
#                 10. 차트는 모두 영어로 표현해 주세요.
#                 </instruction>

#                 <consideration>
#                 1. 사용자가 제공한 데이터셋의 구조와 크기에 따라 코드를 최적화하세요.
#                 2. 복잡한 분석 요청의 경우, 단계별로 접근하여 중간 결과를 확인할 수 있도록 코드를 구성하세요.
#                 3. 데이터의 특성에 따라 적절한 정규화나 스케일링을 고려하세요.
#                 4. 대규모 데이터셋의 경우 성능을 고려하여 코드를 작성하세요.
#                 5. "plt.style.use('seaborn')" 코드는 사용하지 마세요.
#                 6. python의 string code 수행방법(exec())을 사용하려고 합니다. "unterminated string literal" 에러가 발생하지 않게 코드를 작성하세요.\n
#                 7. 코드가 길어 다음 라인에 연속해서 작성해야 하는 경우, backslash(\)를 사용하여 라인을 연결하세요.
#                 8. 이 지침을 따라 사용자의 요청에 맞는 정확하고 효과적인 차트 생성 코드를 작성하고, JSON 형식으로 출력하세요.
#                 9. 차트는 show()함수를 통해 시각화하며, "./output/chart.png"로 저장하고, 경로는 output_format에 맞춰 저장하세요.
#                 10. 만약 코드 수행에 대한 에러(<error_log>가 주어질 경우, 에러를 고려해서 코드를 수정하세요.
#                 </consideration>
#                 '''
#             )

#             system_prompts = bedrock_utils.get_system_prompt(system_prompts=system_prompts)

#             user_prompts = dedent(
#                 '''
#                 This is the result of `print(df.head())`: <dataset>{dataset}</dataset>
#                 Here is the column information in detail, this is the results of `print(column_info)`: <column_info>{column_info}</column_info>
#                 Here is the question: <ask>{ask}</ask>
#                 Here is the error log: <error_log>{error_log}</error_log>
#                 Variable `df: pd.DataFrame` is already declared.
                
#                 '''
#             )

#             context = {
#                 "dataset": str(self.df.sample(10, random_state=0).to_csv()),
#                 "column_info": str(self.column_info.to_csv()),
#                 "ask": ask_reformulation,
#                 "error_log": "None" if code_error == "None" else code_error
#             }
#             user_prompts = user_prompts.format(**context)

#             message = self._get_message_from_string(role="user", string=user_prompts)            
#             self.messages.append(message)

#             resp, messages_updated = self.llm_caller.invoke(messages=self.messages, system_prompts=system_prompts, llm_name="sonnet")
#             self.messages = messages_updated

#             results = eval(resp['text'])
#             code, img_path = results["code"], results["img_path"]

#             self.timer.measure("node: code_generation_for_chart")
#             self.timer.print_measurements()

#             return self.state(code=code, img_path=img_path, prev_node="CODE_GENERATION")

#         def chart_generation(state):

#             print("---CHART GENERATION---")
#             df, code = self.df, state["code"]

#             try:
#                 results = exec(code, {"df": df})
#                 return self.state(code_err="None", prev_node="CHART_GENERATION")
                
#             except Exception as e:
#                 error_type = type(e).__name__
#                 error_message = str(e)
#                 error_traceback = traceback.format_exc()

#                 error = f"Error Type: {error_type}\nError Message: {error_message}\n\nTraceback:\n{error_traceback}"
#                 print (f"error: {error}")
#                 return self.state(code_err=error, prev_node="CHART_GENERATION")

#         def code_checker(state):

#             print("---CODE CHECKER---")
#             code_error = state["code_err"]

#             if code_error == "None":
#                 print ("---GO TO CHART DESCRIPTION---")
#                 return "continue"
#             else:
#                 print ("---[ERROR] GO TO CODE REWRITE---")
#                 return "rewrite"
            
#         def chart_description(state):

#             print("---CHART DESCRIPTION---")
#             img_path = state["img_path"] # PNG 파일 경로

#             system_prompts = dedent(
#                 '''
#                 <task>
#                  사용자의 요청(ask)에 따라 생성된 차트(PNG 형식)를 분석하고 설명합니다. 사용자의 원래 요청을 고려하여 차트의 내용을 정확하고 상세하게 해석하고, 관련 인사이트를 제공합니다.
#                  </task>
                 
#                 <output_format>
#                 다음 정보를 포함하여 응답하세요:
#                 1. 차트 개요: 차트 유형과 전반적인 구조 설명
#                 2. 데이터 분석: 주요 데이터 포인트, 추세, 패턴 설명
#                 3. 사용자 요청 연관성: 차트가 사용자의 요청을 어떻게 충족시키는지 설명
#                 4. 주요 인사이트: 차트에서 도출할 수 있는 중요한 결론이나 통찰
#                 5. 한계점 및 추가 고려사항: 차트의 제한사항이나 추가 분석 필요성
#                 6. 요약 및 결론: 분석의 핵심 포인트와 사용자 요청에 대한 직접적인 답변
#                 </output_format>
                
#                 <instruction>
#                 1. 사용자의 요청(ask) 분석:
#                     - 사용자가 얻고자 하는 정보와 주요 키워드 파악
#                 2. 차트 유형 식별:
#                     - 차트 유형 파악 및 사용자 요청과의 적절성 평가
#                 3. 데이터 분석:
#                     - 주요 데이터 포인트, 추세, 패턴, 이상치 관찰
#                     - 관련 통계 정보 파악 (최대값, 최소값, 평균 등)
#                 4. 차트 구성 요소 설명:
#                     - x축, y축, 범례, 제목, 라벨 등의 의미 해석
#                 5. 사용자 요청과의 연관성 설명:
#                     - 차트가 사용자 요청을 어떻게 충족시키는지 구체적으로 설명
#                 6. 인사이트 도출:
#                     - 차트에서 볼 수 있는 주요 인사이트나 결론 제시
#                     - 데이터의 의미를 사용자 요청 맥락에서 해석
#                 7. 한계점 및 추가 고려사항 언급:
#                     - 차트의 한계점이나 누락된 정보 지적
#                     - 추가 분석이나 데이터 필요성 제안
#                 8. 요약 및 결론 제시:
#                     - 분석의 핵심 포인트 요약
#                     - 사용자의 원래 요청에 대한 직접적인 답변 제공
#                 </instruction>
                
#                 <consideration>
#                 1. 객관적이고 중립적인 톤을 유지하며, 데이터에 기반한 설명 제공
#                 2. 전문 용어 사용 시 필요에 따라 간단한 설명 추가
#                 3. 사용자의 추가 질문 가능성을 고려하여 상세한 설명이 필요한 부분 명시
#                 4. 차트나 데이터의 품질 문제가 있을 경우 적절히 지적
#                 5. 사용자의 요청과 관련성이 낮은 차트 세부사항은 간략히 다루거나 생략
#                 6. 시각적 요소(색상, 크기 등)가 데이터 해석에 중요한 경우 이를 언급
#                 7. 가능한 경우, 차트에서 얻은 정보를 실제 상황이나 의사결정에 적용하는 방법 제안
#                 8. 차트가 표현하는 데이터의 출처나 시간 범위가 중요한 경우 이를 강조
#                 9. chart description 생성 시 '"' 사용하지 말 것. 
#                 </consideration>
#                 '''
#              )

#             system_prompts = bedrock_utils.get_system_prompt(system_prompts=system_prompts)

#             user_prompts = dedent(
#                 '''
#                 Here is the question: <ask>{ask}</ask>
#                 Here is chart: 
#                 '''
#             )

#             context = {
#                 "ask": ask_reformulation
#             }
#             user_prompts = user_prompts.format(**context)
            
#             self.img_bytes, img_base64 = self._png_to_bytes(img_path)
#             message = self._get_message_from_string(role="user", string=user_prompts, img=self.img_bytes)
#             self.messages.append(message)

#             resp, messages_updated = self.llm_caller.invoke(messages=self.messages, system_prompts=system_prompts, llm_name="sonnet")
#             self.messages = messages_updated
#             chart_description = self._get_string_from_message(self.messages[-1])

#             self.timer.measure("node: chart_description")
#             self.timer.print_measurements()
             
#             return self.state(chart_desc=chart_description, prev_node="CHART_DESCRIPTION")
            
#         # langgraph.graph에서 StateGraph와 END를 가져옵니다.
#         workflow = StateGraph(self.state)

#         # Todo 를 작성합니다.
#         workflow.add_node("agent", agent)  # 에이전트 노드를 추가합니다.
#         workflow.add_node("ask_reformulation", ask_reformulation)  # 요청을 차트생성에 용이하게 수정하는 노드를 추가합니다.
#         workflow.add_node("code_generation_for_chart", code_generation_for_chart)  # 차트 생성을 위한 코드 생성 노드를 추가합니다.
#         workflow.add_node("chart_generation", chart_generation)  # 생성된 코드를 실행하여 노드를 생성하는 노드를 추가합니다.
#         workflow.add_node("chart_description", chart_description)  # 생성된 코드를 설명하는 노드를 추가합니다.
        
#         # 각 노드들을 연결합니다.
#         workflow.add_conditional_edges(
#             "agent",
#             # 에이전트 결정 평가
#             should_chart_generation,
#             {
#                 # 도구 노드 호출
#                 "continue": "ask_reformulation",
#                 "end": END,
#             },
#         )
#         workflow.add_edge("ask_reformulation", "code_generation_for_chart")
#         workflow.add_edge("code_generation_for_chart", "chart_generation")
#         workflow.add_conditional_edges(
#             "chart_generation",
#             # 에이전트 결정 평가
#             code_checker,
#             {
#                 # 도구 노드 호출
#                 "continue": "chart_description",
#                 "rewrite": "code_generation_for_chart",
#             },
#         )
#         #workflow.add_edge("chart_generation", "chart_description")
#         workflow.add_edge("chart_description", END)

#         # 시작점을 설정합니다.
#         workflow.set_entry_point("agent")

#         # 기록을 위한 메모리 저장소를 설정합니다.
#         memory = MemorySaver()

#         # 그래프를 컴파일합니다.
#         self.app = workflow.compile(checkpointer=memory)        
#         self.config = RunnableConfig(recursion_limit=100, configurable={"thread_id": "Text2Chart"})

#     def invoke(self, **kwargs):
        
#         inputs = self.state(ask=kwargs["ask"])
#         # app.stream을 통해 입력된 메시지에 대한 출력을 스트리밍합니다.
#         for output in self.app.stream(inputs, self.config):
#             # 출력된 결과에서 키와 값을 순회합니다.
#             for key, value in output.items():
#                 # 노드의 이름과 해당 노드에서 나온 출력을 출력합니다.
#                 pprint.pprint(f"\nOutput from node '{key}':")
#                 pprint.pprint("---")
#                 # 출력 값을 예쁘게 출력합니다.
#                 pprint.pprint(value, indent=2, width=80, depth=None)
#             # 각 출력 사이에 구분선을 추가합니다.
#             pprint.pprint("\n---\n")
    
#     def show_graph(self, ):
        
#         from IPython.display import Image, display

#         try:
#             display(
#                 Image(self.app.get_graph(xray=True).draw_mermaid_png())
#             )  # 실행 가능한 객체의 그래프를 mermaid 형식의 PNG로 그려서 표시합니다. 
#             # xray=True는 추가적인 세부 정보를 포함합니다.
#         except:
#             # 이 부분은 추가적인 의존성이 필요하며 선택적으로 실행됩니다.
#             pass

            

In [ ]:
class video_analyzer():
    
    def __init__(self, **kwargs):

        system_prompt = kwargs["system_prompt"]
        self.system_prompt = self._get_message_from_string(role="system", string=system_prompt)
        
        human_prompt=kwargs["human_prompt"]
        self.human_prompt = self._get_message_from_string(role="human", string=human_prompt)
        
        
        self.llm_text = kwargs["llm_text"]        
        self.retriever = kwargs["retriever"]
        
        self.return_context = kwargs.get("return_context", False)
        self.verbose = kwargs.get("verbose", False)
                     
    def _get_message_from_string(self, role, string):
        
        if role == "system": message= SystemMessagePromptTemplate.from_template(string)
        elif role == "human": message= HumanMessagePromptTemplate.from_template(string)
        elif role == "ai": message = AIMessage(content=string)
            
        return message
        
        
    def invoke(self, **kwargs):
               
        query, verbose = kwargs["query"], kwargs.get("verbose", self.verbose)
        tables, images = None, None
        
        print ("verbose", verbose)
        
        if self.retriever.complex_doc:
            retrieval, tables, images = self.retriever.invoke(query)

            invoke_args = {
                "contexts": "\n\n".join([doc.page_content for doc in retrieval]),
                "tables_text": "\n\n".join([doc.page_content for doc in tables]),
                "tables_html": "\n\n".join([doc.metadata["text_as_html"] if "text_as_html" in doc.metadata else "" for doc in tables]),
                "question": query
            }
            human_prompt_complex_doc = prompt_repo.get_human_prompt(images=images, tables=tables)
            human_prompt_complex_doc = self._get_message_from_string(role="human", string=human_prompt_complex_doc)
            prompt = ChatPromptTemplate([self.system_prompt, human_prompt_complex_doc])
                
            self.chain = prompt | self.llm_text | StrOutputParser()
            
        else:
            retrieval = self.retriever.invoke(query)
            invoke_args = {
                "contexts": "\n\n".join([doc.page_content for doc in retrieval]),
                "question": query
            }
            prompt = ChatPromptTemplate([self.system_prompt, self.human_prompt])
            self.chain = prompt | self.llm_text | StrOutputParser()
            
        # Invoke the chain
        stream = self.chain.stream(
            invoke_args,
            config={'callbacks': [ConsoleCallbackHandler()]} if verbose else {}
        )
            
        response = ""
        for chunk in stream: response += chunk

        
        return response, retrieval if self.return_context else response

In [ ]:
import cv2
import matplotlib.pyplot as plt

- cv2.CAP_PROP_FPS: 초당 프레임 수
- cv2.CAP_PROP_FRAME_COUNT: 총 프레임 수
- cv2.CAP_PROP_FRAME_WIDTH: 프레임 너비
- cv2.CAP_PROP_FRAME_HEIGHT: 프레임 높이
- cv2.CAP_PROP_POS_FRAMES: 현재 프레임 번호
- cv2.CAP_PROP_POS_MSEC: 현재 위치(밀리초)
- cv2.CAP_PROP_FOURCC: 비디오 코덱
- cv2.CAP_PROP_BRIGHTNESS: 밝기
- cv2.CAP_PROP_CONTRAST: 대비

In [ ]:
video_path = "./video/video_sample.mp4"

In [ ]:
import cv2
import os
import shutil
from typing import Tuple, Optional

def setup_directory(dir_path):
    
    print ("===setup_directory===")
    # 디렉토리가 존재하는지 확인
    if os.path.exists(dir_path):
        # 존재하면 삭제
        shutil.rmtree(dir_path)
        print(f"기존 디렉토리 삭제됨: {dir_path}")
    
    # 디렉토리 생성
    os.makedirs(dir_path)
    print(f"디렉토리 생성됨: {dir_path}")
    print ("=====================")

def sample_video_frames(video_path: str, sample_msec: int, output_dir: Optional[str] = None) -> Tuple[int, int]:
    
    print ("===sample_video_frames===")
    """
    비디오에서 특정 시간 간격으로 프레임을 샘플링하는 함수
    
    Args:
        video_path (str): 비디오 파일 경로
        sample_msec (int): 샘플링 간격 (밀리초)
        output_dir (Optional[str]): 프레임 저장 디렉토리. None이면 저장하지 않음
    
    Returns:
        Tuple[int, int]: (총 프레임 수, 샘플링된 프레임 수)
    
    Raises:
        FileNotFoundError: 비디오 파일이 없는 경우
        ValueError: 샘플링 간격이 잘못된 경우
    """
    # 입력값 검증
    if not os.path.exists(video_path):
        raise FileNotFoundError(f"비디오 파일을 찾을 수 없습니다: {video_path}")
    
    if sample_msec <= 0:
        raise ValueError("샘플링 간격은 0보다 커야 합니다")
    
    # 비디오 캡처 객체 생성
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise RuntimeError("비디오 파일을 열 수 없습니다")
    
    # 비디오 정보 가져오기
    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    # 샘플링 간격(프레임 단위) 계산
    frame_interval = max(1, int(sample_msec / 1000 * fps))
    
    # 출력 디렉토리 생성 (지정된 경우)
    if output_dir is not None:
        setup_directory(output_dir)
    
    sampled_count = 0
    frame_count = 0
    sampled_frame = []
    
    while True:
        ret, frame = cap.read()
        if not ret: break
            
        # frame_interval마다 프레임 처리
        if frame_count % frame_interval == 0:
            if output_dir is not None:
                # 프레임 저장
                frame_path = os.path.join(output_dir, f"frame_{frame_count:06d}.jpg")
                cv2.imwrite(frame_path, frame)
            sampled_count += 1
            sampled_frame.append(frame)
        frame_count += 1
        
        # 진행상황 출력 (10% 단위)
        if frame_count % (total_frames // 10) == 0:
            progress = (frame_count / total_frames) * 100
            print(f"진행률: {progress:.1f}%")
    
    # 자원 해제
    cap.release()
    
    print(f"\n처리 완료:")
    print(f"총 프레임 수: {total_frames}")
    print(f"프레임 크기: {sampled_frame[0].shape[1]}X{sampled_frame[0].shape[0]}")
    print(f"샘플링된 프레임 수: {sampled_count}")
    print(f"샘플링 간격: {frame_interval}프레임 ({sample_msec}ms)")
    if output_dir is not None:
        print(f"저장 위치: {output_dir}")
    
    print ("=========================")
    
    return sampled_frame, total_frames, sampled_count

In [ ]:
# 프레임 저장하면서 샘플링
sampled_frames, total_frame_cnt, sampled_cnt = sample_video_frames(
    video_path="./video/video_sample.mp4",
    sample_msec=1000,  # 100ms(0.1초)마다 샘플링
    output_dir="./workplace"
)

In [ ]:
sampled_frames[0]

In [ ]:
frames[0].shape

In [ ]:
rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # 이미지 표시
        plt.figure(figsize=(10, 6))
        plt.imshow(rgb_frame)
        plt.axis('off')  # 축 숨기기
        plt.show()

In [ ]:
class TimeMeasurement:
    def __init__(self):
        self.start_time = None
        self.measurements = {}

    def start(self):
        self.start_time = time.time()

    def measure(self, section_name):
        if self.start_time is None:
            raise ValueError("start() 메서드를 먼저 호출해야 합니다.")
        
        end_time = time.time()
        elapsed_time = end_time - self.start_time
        self.measurements[section_name] = elapsed_time
        self.start_time = end_time  # 다음 구간 측정을 위해 시작 시간 재설정

    def reset(self, ):
        self.measurements = {}

    def print_measurements(self):
        for section, elapsed_time in self.measurements.items():
            #print(f"{section}: {elapsed_time:.5f} 초")
            print(colored (f"\nelapsed time: {section}: {elapsed_time:.5f} 초", "red"))

### 3.1 Agent state 

In [ ]:
class GraphState(TypedDict):
    ask: list[str]
    target_apps: list[str]
    ask_refo: str
    code: str
    code_err: str
    img_path: str
    img_bytes: str
    chart_desc: str
    prev_node: str

In [ ]:
class genai_analyzer():

    def __init__(self, **kwargs):

        self.llm_sonnet=kwargs["llm_sonnet"]
        self.llm_haiku=kwargs["llm_haiku"]
        self.df = kwargs["df"]
        self.column_info = kwargs["column_info"]
        self.state = GraphState

        self.llm_caller = llm_call(
            llm_sonnet=self.llm_sonnet,
            llm_haiku=self.llm_haiku,
            verbose=False
        ) 

        self._graph_definition()
        self.messages = []
        self.img_bytes = ""

        self.timer = TimeMeasurement()

    def _get_string_from_message(self, message):
        return message["content"][0]["text"]

    def _get_message_from_string(self, role, string, img=None):
        
        message = {
            "role": role,
            "content": [{"text": dedent(string)}]
        }
        
        if img is not None:
            img_message = {
                "image": {
                    "format": 'png',
                    "source": {"bytes": img}
                }
            }
            message["content"].append(img_message)

        return message

    def _png_to_bytes(self, file_path):
        try:
            with open(file_path, "rb") as image_file:
                # 파일을 바이너리 모드로 읽기
                binary_data = image_file.read()
                
                # 바이너리 데이터를 base64로 인코딩
                base64_encoded = base64.b64encode(binary_data)
                
                # bytes 타입을 문자열로 디코딩
                base64_string = base64_encoded.decode('utf-8')
                
                return binary_data, base64_string
                
        except FileNotFoundError:
            return "Error: 파일을 찾을 수 없습니다."
        except Exception as e:
            return f"Error: {str(e)}"

    def show_image(base64_string):
        try:
            # base64 문자열을 디코딩하여 바이너리 데이터로 변환
            image_data = base64.b64decode(base64_string)
            
            # 바이너리 데이터를 이미지로 변환
            image = Image.open(io.BytesIO(image_data))
            
            # matplotlib을 사용하여 이미지 표시
            plt.imshow(image)
            plt.axis('off')  # 축 제거
            plt.show()
        except Exception as e:
            print(f"Error: 이미지를 표시하는 데 실패했습니다. {str(e)}")

    def get_messages(self, ):
        return self.messages
        
    def _graph_definition(self, **kwargs):

        def agent(state):

            self.timer.start()
            self.timer.reset()

            print("---CALL AGENT---")
            ask = state["ask"]

            """
            현재 상태를 기반으로 에이전트 모델을 호출하여 응답을 생성합니다. 질문에 따라 검색 도구를 사용하여 검색을 결정하거나 단순히 종료합니다.
        
            Args:
                state (messages): 현재 상태
        
            Returns:
                state (messages): 현재 상태 메시지에 에이전트 응답이 추가된 업데이트된 상태
            """

            system_prompts = dedent(
                '''
                <task>
                사용자 메시지를 분석하여 차트 생성 여부를 결정하는 에이전트 역할 수행
                </task>
                
                <instruction>
                1. 사용자 메시지를 주의 깊게 분석하세요.
                2. 차트, 그래프, 데이터 시각화와 관련된 키워드를 찾으세요.
                3. 수치 데이터나 통계 정보의 존재 여부를 확인하세요.
                4. 분석 결과를 바탕으로 차트 생성 필요성을 판단하세요.
                5. 주어진 데이터 (dataset) 및 컬럼 정보 (column_info)를 참고하여 생성 가능 여부 또한 고려하세요.
                6. 데이터로부터 답변할 수 없는 요청을 한다면 최종 결정을 "END"로 하세요.
                7. 판단이 모호한 경우, 사용자에게 직접 차트 생성 의도를 물어보세요.
                </instruction>
                
                <consideration>
                - 사용자의 의도를 정확히 파악하는 것이 중요합니다.
                - 명시적인 차트 요청이 없더라도 데이터 시각화가 유용할 수 있는 상황을 고려하세요.
                - 단순한 질문이나 대화 종료 요청은 차트 생성이 불필요할 수 있습니다.
                - 기존 요청 결과에 대한 추가사항이라고 판단되면, 추가 코드 생성을 하지 말고 "GENERATE_CHART"를 출력하세요.
                </consideration>
                
                <output_format>
                결정에 따라 다음 중 하나를 출력하세요. 반드시 아래 2개중 1개를 선택합니다.:
                1. "GENERATE_CHART (간단한 이유)" - 차트 생성이 필요한 경우
                2. "END (간단한 이유)" - 차트 생성이 할 수 없거나 대화를 종료해야 하는 경우
                
                예시:
                GENERATE_CHART (사용자가 연간 수익 추이 그래프 요청)
                END (단순한 날씨 질문으로 차트 불필요)
                </output_format>

                This is the <request>{request}</request>
                
                '''
            )

            context = {
                "request": str(self.df.sample(10, random_state=0).to_csv()),
                "column_info": str(self.column_info.to_csv())
            }
            system_prompts = system_prompts.format(**context)
            system_prompts = bedrock_utils.get_system_prompt(system_prompts=system_prompts)

            

            message = self._get_message_from_string(role="user", string=ask)
            self.messages.append(message)

            resp, messages_updated = self.llm_caller.invoke(messages=self.messages, system_prompts=system_prompts, llm_name="sonnet")
            self.messages = messages_updated
            
            return self.state(ask=ask, prev_node="AGENT")

        def should_chart_generation(state):
            """
            에이전트가 차트를 생성하는데 있어 추가적으로 고려해야 하는 상황이 있는지 결정합니다.
        
            이 함수는 상태의 마지막 메시지에서 함수 호출을 확인합니다. 함수 호출이 있으면 정보 검색 프로세스를 계속합니다. 그렇지 않으면 프로세스를 종료합니다.
        
            Args:
                state (messages): 현재 상태
        
            Returns:
                str: 검색 프로세스를 "계속"하거나 "종료"하는 결정
            """
        
            print("\n---DECIDE TO CHART GENERATION---")
            #messages = state["messages"]
            last_message = self._get_string_from_message(self.messages[-1])
            
            # 함수 호출이 없으면 종료합니다.
            if "GENERATE_CHART" not in last_message:
                print("---DECISION: DO NOT CHART GENERATION / DONE---")
                return "end"
            # 그렇지 않으면 함수 호출이 있으므로 계속합니다.
            else:
                print("---DECISION: CHART GENERATION---")
                return "continue"

        def ask_reformulation(state):

            print("---ASK REFORMULATION---")
            ask = state["ask"]

            system_prompts = dedent(
                '''
                당신은 사용자의 텍스트 요청을 분석하여 중요한 정보를 추출하는 전문가입니다.
                주어진 structured dataset(df)에 대한 분석 요청(request)을 처리하는 것이 당신의 주요 임무입니다.

                <task>
                1. 사용자의 텍스트 요청에서 분석 대상이 되는 target app의 이름을 식별하고 추출하세요.
                2. 사용자의 구체적인 분석 요청 사항을 분석하고, 필요하다면 결과를 차트로 표현하기 적합현 형태로 요청 사항을 수정해 주세요.
                </task>
                
                <output_format>
                JSON 형식으로 다음 정보를 포함하여 응답하세요:
                {{
                  "target_apps": ["추출된 앱 이름"],
                  "ask_reformulation": "파악된 분석 요청 사항"
                }}
                </output_format>

                <instruction>
                - target app이 명시적으로 언급되지 않은 경우, "target_app" 필드를 "unspecified"로 설정하세요.
                - target app이 복수 개인 경우, list 형태로 모두 언급하세요. 예를 들자면 ["앱 이름 1", "앱 이름 2"]로 표현합니다. 
                - 분석 요청이 불명확한 경우, 가능한 한 사용자의 의도를 추론하여 "ask_reformulation" 필드를 작성하세요.
                - 추출 및 파악한 정보만을 간결하게 제공하고, 추가적인 설명이나 해석은 하지 마세요.
                </instruction>

                이 정보를 바탕으로 다음 노드가 적절한 분석을 수행할 수 있도록 정확하고 명확한 정보를 제공하는 것이 중요합니다.
                '''
            )
            system_prompts = bedrock_utils.get_system_prompt(system_prompts=system_prompts)

            user_prompts = dedent(
                '''
                This is the result of `print(df.head())`: <dataset>{dataset}</dataset>
                Here is the column information in detail, this is the results of `print(column_info)`: <column_info>{column_info}</column_info>
                Here is user's request: <request>{ask}</request>
                '''
            )
            context = {
                "dataset": str(self.df.sample(10, random_state=0).to_csv()),
                "column_info": str(self.column_info.to_csv()),
                "ask": ask
            }
            user_prompts = user_prompts.format(**context)
            
            message = self._get_message_from_string(role="user", string=user_prompts)            
            self.messages.append(message)

            resp, messages_updated = self.llm_caller.invoke(messages=self.messages, system_prompts=system_prompts, llm_name="sonnet")

            results = eval(resp['text'])
            target_apps, ask_reformulation = results["target_apps"], results["ask_reformulation"]
            self.messages=messages_updated

            return self.state(target_apps=target_apps, ask_refo=ask_reformulation, prev_node="ASK_REFORMULATION")

        def code_generation_for_chart(state):

            print("---CODE GENERATION FOR CHART---")
            ask_reformulation = state["ask_refo"]
            previous_node = state["prev_node"]
            code_error = state["code_err"]

            system_prompts = dedent(
                '''
                당신은 데이터 분석과 시각화 전문가입니다.
                주어진 structured dataset, dataset의 컬럼 정보, 그리고 사용자의 분석 요청사항을 바탕으로 적절한 차트를 생성하는 Python 코드를 작성하는 것이 당신의 임무입니다.

                <task>
                사용자의 요청에 적합한 차트생성 python 코드 작성
                </task>

                <input>
                1. dataset: 분석할 데이터셋
                2. column_info: 각 컬럼의 이름과 데이터 타입
                3. question: 어떤 분석을 원하는지에 대한 설명
                </input>
                
                <output_format>
                JSON 형식으로 다음 형태로 응답하세요. 절대 JSON 포멧 외 텍스트는 넣지 마세요.:
                {{
                    "code": """사용자의 요청을 충족시키는 차트를 생성하는 Python 코드"""
                    "img_path": """생성된 차트의 저장 경로"""
                }}
                </output_format>

                <instruction>
                1. 데이터셋과 컬럼 정보를 신중히 분석하세요.
                2. 사용자의 분석 요청사항을 정확히 이해하세요.
                3. 요청사항에 가장 적합한 차트 유형을 선택하세요 (예: 막대 그래프, 선 그래프, 산점도, 파이 차트 등).
                4. 선택한 차트 유형에 맞는 Python 라이브러리를 사용하세요 (예: matplotlib, seaborn, plotly 등).
                5. 데이터 전처리가 필요한 경우 pandas를 사용하여 데이터를 적절히 가공하세요.
                6. 차트의 제목, 축 레이블, 범례 등을 명확하게 설정하세요.
                7. 필요한 경우 차트의 색상, 스타일, 크기 등을 조정하여 가독성을 높이세요.
                8. 코드 실행 시 발생할 수 있는 예외 상황을 고려하여 적절한 예외 처리를 포함하세요.
                9. 생성된 차트를 저장하거나 표시하는 코드를 포함하세요.
                10. 차트는 모두 영어로 표현해 주세요.
                </instruction>

                <consideration>
                1. 사용자가 제공한 데이터셋의 구조와 크기에 따라 코드를 최적화하세요.
                2. 복잡한 분석 요청의 경우, 단계별로 접근하여 중간 결과를 확인할 수 있도록 코드를 구성하세요.
                3. 데이터의 특성에 따라 적절한 정규화나 스케일링을 고려하세요.
                4. 대규모 데이터셋의 경우 성능을 고려하여 코드를 작성하세요.
                5. "plt.style.use('seaborn')" 코드는 사용하지 마세요.
                6. python의 string code 수행방법(exec())을 사용하려고 합니다. "unterminated string literal" 에러가 발생하지 않게 코드를 작성하세요.\n
                7. 코드가 길어 다음 라인에 연속해서 작성해야 하는 경우, backslash(\)를 사용하여 라인을 연결하세요.
                8. 이 지침을 따라 사용자의 요청에 맞는 정확하고 효과적인 차트 생성 코드를 작성하고, JSON 형식으로 출력하세요.
                9. 차트는 show()함수를 통해 시각화하며, "./output/chart.png"로 저장하고, 경로는 output_format에 맞춰 저장하세요.
                10. 만약 코드 수행에 대한 에러(<error_log>가 주어질 경우, 에러를 고려해서 코드를 수정하세요.
                </consideration>
                '''
            )

            system_prompts = bedrock_utils.get_system_prompt(system_prompts=system_prompts)

            user_prompts = dedent(
                '''
                This is the result of `print(df.head())`: <dataset>{dataset}</dataset>
                Here is the column information in detail, this is the results of `print(column_info)`: <column_info>{column_info}</column_info>
                Here is the question: <ask>{ask}</ask>
                Here is the error log: <error_log>{error_log}</error_log>
                Variable `df: pd.DataFrame` is already declared.
                
                '''
            )

            context = {
                "dataset": str(self.df.sample(10, random_state=0).to_csv()),
                "column_info": str(self.column_info.to_csv()),
                "ask": ask_reformulation,
                "error_log": "None" if code_error == "None" else code_error
            }
            user_prompts = user_prompts.format(**context)

            message = self._get_message_from_string(role="user", string=user_prompts)            
            self.messages.append(message)

            resp, messages_updated = self.llm_caller.invoke(messages=self.messages, system_prompts=system_prompts, llm_name="sonnet")
            self.messages = messages_updated

            results = eval(resp['text'])
            code, img_path = results["code"], results["img_path"]

            self.timer.measure("node: code_generation_for_chart")
            self.timer.print_measurements()

            return self.state(code=code, img_path=img_path, prev_node="CODE_GENERATION")

        def chart_generation(state):

            print("---CHART GENERATION---")
            df, code = self.df, state["code"]

            try:
                results = exec(code, {"df": df})
                return self.state(code_err="None", prev_node="CHART_GENERATION")
                
            except Exception as e:
                error_type = type(e).__name__
                error_message = str(e)
                error_traceback = traceback.format_exc()

                error = f"Error Type: {error_type}\nError Message: {error_message}\n\nTraceback:\n{error_traceback}"
                print (f"error: {error}")
                return self.state(code_err=error, prev_node="CHART_GENERATION")

        def code_checker(state):

            print("---CODE CHECKER---")
            code_error = state["code_err"]

            if code_error == "None":
                print ("---GO TO CHART DESCRIPTION---")
                return "continue"
            else:
                print ("---[ERROR] GO TO CODE REWRITE---")
                return "rewrite"
            
        def chart_description(state):

            print("---CHART DESCRIPTION---")
            img_path = state["img_path"] # PNG 파일 경로

            system_prompts = dedent(
                '''
                <task>
                 사용자의 요청(ask)에 따라 생성된 차트(PNG 형식)를 분석하고 설명합니다. 사용자의 원래 요청을 고려하여 차트의 내용을 정확하고 상세하게 해석하고, 관련 인사이트를 제공합니다.
                 </task>
                 
                <output_format>
                다음 정보를 포함하여 응답하세요:
                1. 차트 개요: 차트 유형과 전반적인 구조 설명
                2. 데이터 분석: 주요 데이터 포인트, 추세, 패턴 설명
                3. 사용자 요청 연관성: 차트가 사용자의 요청을 어떻게 충족시키는지 설명
                4. 주요 인사이트: 차트에서 도출할 수 있는 중요한 결론이나 통찰
                5. 한계점 및 추가 고려사항: 차트의 제한사항이나 추가 분석 필요성
                6. 요약 및 결론: 분석의 핵심 포인트와 사용자 요청에 대한 직접적인 답변
                </output_format>
                
                <instruction>
                1. 사용자의 요청(ask) 분석:
                    - 사용자가 얻고자 하는 정보와 주요 키워드 파악
                2. 차트 유형 식별:
                    - 차트 유형 파악 및 사용자 요청과의 적절성 평가
                3. 데이터 분석:
                    - 주요 데이터 포인트, 추세, 패턴, 이상치 관찰
                    - 관련 통계 정보 파악 (최대값, 최소값, 평균 등)
                4. 차트 구성 요소 설명:
                    - x축, y축, 범례, 제목, 라벨 등의 의미 해석
                5. 사용자 요청과의 연관성 설명:
                    - 차트가 사용자 요청을 어떻게 충족시키는지 구체적으로 설명
                6. 인사이트 도출:
                    - 차트에서 볼 수 있는 주요 인사이트나 결론 제시
                    - 데이터의 의미를 사용자 요청 맥락에서 해석
                7. 한계점 및 추가 고려사항 언급:
                    - 차트의 한계점이나 누락된 정보 지적
                    - 추가 분석이나 데이터 필요성 제안
                8. 요약 및 결론 제시:
                    - 분석의 핵심 포인트 요약
                    - 사용자의 원래 요청에 대한 직접적인 답변 제공
                </instruction>
                
                <consideration>
                1. 객관적이고 중립적인 톤을 유지하며, 데이터에 기반한 설명 제공
                2. 전문 용어 사용 시 필요에 따라 간단한 설명 추가
                3. 사용자의 추가 질문 가능성을 고려하여 상세한 설명이 필요한 부분 명시
                4. 차트나 데이터의 품질 문제가 있을 경우 적절히 지적
                5. 사용자의 요청과 관련성이 낮은 차트 세부사항은 간략히 다루거나 생략
                6. 시각적 요소(색상, 크기 등)가 데이터 해석에 중요한 경우 이를 언급
                7. 가능한 경우, 차트에서 얻은 정보를 실제 상황이나 의사결정에 적용하는 방법 제안
                8. 차트가 표현하는 데이터의 출처나 시간 범위가 중요한 경우 이를 강조
                9. chart description 생성 시 '"' 사용하지 말 것. 
                </consideration>
                '''
             )

            system_prompts = bedrock_utils.get_system_prompt(system_prompts=system_prompts)

            user_prompts = dedent(
                '''
                Here is the question: <ask>{ask}</ask>
                Here is chart: 
                '''
            )

            context = {
                "ask": ask_reformulation
            }
            user_prompts = user_prompts.format(**context)
            
            self.img_bytes, img_base64 = self._png_to_bytes(img_path)
            message = self._get_message_from_string(role="user", string=user_prompts, img=self.img_bytes)
            self.messages.append(message)

            resp, messages_updated = self.llm_caller.invoke(messages=self.messages, system_prompts=system_prompts, llm_name="sonnet")
            self.messages = messages_updated
            chart_description = self._get_string_from_message(self.messages[-1])

            self.timer.measure("node: chart_description")
            self.timer.print_measurements()
             
            return self.state(chart_desc=chart_description, prev_node="CHART_DESCRIPTION")
            
        # langgraph.graph에서 StateGraph와 END를 가져옵니다.
        workflow = StateGraph(self.state)

        # Todo 를 작성합니다.
        workflow.add_node("agent", agent)  # 에이전트 노드를 추가합니다.
        workflow.add_node("ask_reformulation", ask_reformulation)  # 요청을 차트생성에 용이하게 수정하는 노드를 추가합니다.
        workflow.add_node("code_generation_for_chart", code_generation_for_chart)  # 차트 생성을 위한 코드 생성 노드를 추가합니다.
        workflow.add_node("chart_generation", chart_generation)  # 생성된 코드를 실행하여 노드를 생성하는 노드를 추가합니다.
        workflow.add_node("chart_description", chart_description)  # 생성된 코드를 설명하는 노드를 추가합니다.
        
        # 각 노드들을 연결합니다.
        workflow.add_conditional_edges(
            "agent",
            # 에이전트 결정 평가
            should_chart_generation,
            {
                # 도구 노드 호출
                "continue": "ask_reformulation",
                "end": END,
            },
        )
        workflow.add_edge("ask_reformulation", "code_generation_for_chart")
        workflow.add_edge("code_generation_for_chart", "chart_generation")
        workflow.add_conditional_edges(
            "chart_generation",
            # 에이전트 결정 평가
            code_checker,
            {
                # 도구 노드 호출
                "continue": "chart_description",
                "rewrite": "code_generation_for_chart",
            },
        )
        #workflow.add_edge("chart_generation", "chart_description")
        workflow.add_edge("chart_description", END)

        # 시작점을 설정합니다.
        workflow.set_entry_point("agent")

        # 기록을 위한 메모리 저장소를 설정합니다.
        memory = MemorySaver()

        # 그래프를 컴파일합니다.
        self.app = workflow.compile(checkpointer=memory)        
        self.config = RunnableConfig(recursion_limit=100, configurable={"thread_id": "Text2Chart"})

    def invoke(self, **kwargs):
        
        inputs = self.state(ask=kwargs["ask"])
        # app.stream을 통해 입력된 메시지에 대한 출력을 스트리밍합니다.
        for output in self.app.stream(inputs, self.config):
            # 출력된 결과에서 키와 값을 순회합니다.
            for key, value in output.items():
                # 노드의 이름과 해당 노드에서 나온 출력을 출력합니다.
                pprint.pprint(f"\nOutput from node '{key}':")
                pprint.pprint("---")
                # 출력 값을 예쁘게 출력합니다.
                pprint.pprint(value, indent=2, width=80, depth=None)
            # 각 출력 사이에 구분선을 추가합니다.
            pprint.pprint("\n---\n")
    
    def show_graph(self, ):
        
        from IPython.display import Image, display

        try:
            display(
                Image(self.app.get_graph(xray=True).draw_mermaid_png())
            )  # 실행 가능한 객체의 그래프를 mermaid 형식의 PNG로 그려서 표시합니다. 
            # xray=True는 추가적인 세부 정보를 포함합니다.
        except:
            # 이 부분은 추가적인 의존성이 필요하며 선택적으로 실행됩니다.
            pass

            

In [ ]:
import pandas as pd
from langgraph.graph import END, StateGraph

In [ ]:
df = pd.read_csv("./dataset/app_power_consumption.csv")
column_info = pd.read_csv("dataset/column_info.csv")

In [ ]:
analyzer = genai_analyzer(
    llm_sonnet=llm_sonnet,
    llm_haiku=llm_haiku,
    df=df,
    column_info=column_info
)

In [ ]:
analyzer.show_graph()

In [ ]:
analyzer.invoke(
    ask=dedent("앱 a, b, c 소비 전력에 대한 비교 차트")
)

In [ ]:
analyzer.invoke(
    ask=dedent("너무 많다. 2주일만  보여줘")
)

In [ ]:
analyzer.invoke(
    ask=dedent("비교가 어렵네. 막대 그래프로 변환해 줄래?")
)

In [ ]:
analyzer.invoke(
    ask=dedent("전력 사용량이 가장 큰 지점을 표시해줘")
)

In [ ]:
analyzer.invoke(
    ask=dedent("가장 큰 전력을 쓴 날짜도 표시해줘")
)

In [ ]:
analyzer.invoke(
    ask=dedent("앱 e도 추가해 줄래?")
)

In [ ]:
#8. 코드에 주석을 달아 각 단계를 설명하세요. 주석은 "#####"를 이용하세요.

In [ ]:
#코드설명 백업
'''
<task>
 사용자의 요청(ask)에 따라 생성된 차트(PNG 형식)를 분석하고 설명합니다. 사용자의 원래 요청을 고려하여 차트의 내용을 정확하고 상세하게 해석하고, 관련 인사이트를 제공합니다.
 </task>
 
<output_format>
다음 정보를 포함하여 응답하세요:
1. 주요 인사이트: 차트에서 도출할 수 있는 중요한 결론이나 통찰
2. 한계점 및 추가 고려사항: 차트의 제한사항이나 추가 분석 필요성
</output_format>

<instruction>
1. 사용자의 요청(ask) 분석:
    - 사용자가 얻고자 하는 정보와 주요 키워드 파악
2. 차트 유형 식별:
    - 차트 유형 파악 및 사용자 요청과의 적절성 평가
3. 데이터 분석:
    - 주요 데이터 포인트, 추세, 패턴, 이상치 관찰
    - 관련 통계 정보 파악 (최대값, 최소값, 평균 등)
4. 차트 구성 요소 설명:
    - x축, y축, 범례, 제목, 라벨 등의 의미 해석
5. 사용자 요청과의 연관성 설명:
    - 차트가 사용자 요청을 어떻게 충족시키는지 구체적으로 설명
6. 인사이트 도출:
    - 차트에서 볼 수 있는 주요 인사이트나 결론 제시
    - 데이터의 의미를 사용자 요청 맥락에서 해석
7. 한계점 및 추가 고려사항 언급:
    - 차트의 한계점이나 누락된 정보 지적
    - 추가 분석이나 데이터 필요성 제안
8. 요약 및 결론 제시:
    - 분석의 핵심 포인트 요약
    - 사용자의 원래 요청에 대한 직접적인 답변 제공
</instruction>

<consideration>
1. 객관적이고 중립적인 톤을 유지하며, 데이터에 기반한 설명 제공
2. 전문 용어 사용 시 필요에 따라 간단한 설명 추가
3. 사용자의 추가 질문 가능성을 고려하여 상세한 설명이 필요한 부분 명시
4. 차트나 데이터의 품질 문제가 있을 경우 적절히 지적
5. 사용자의 요청과 관련성이 낮은 차트 세부사항은 간략히 다루거나 생략
6. 시각적 요소(색상, 크기 등)가 데이터 해석에 중요한 경우 이를 언급
7. 가능한 경우, 차트에서 얻은 정보를 실제 상황이나 의사결정에 적용하는 방법 제안
8. 차트가 표현하는 데이터의 출처나 시간 범위가 중요한 경우 이를 강조
9. chart description 생성 시 '"' 사용하지 말 것. 
</consideration>
'''